# 强化学习，反正就他了

需要实现的借口：

两个模型的自对弈完成一局游戏

完成n场自对弈后，学习胜者策略

每进行一段时间，检测胜率，将更优的保留下来。

如果不行，使用tensorflow载入别人的牛逼模型（ 开外挂。



代码开头必须的一些库和常量

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Mar 24 12:40:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    28W /  70W |   1205MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from torch.nn.modules.pooling import MaxPool2d
import torch
import torch.nn as nn 
import numpy as np
import random
import copy
import mcts
# 必要的一些库
import json
N = 11
inputc = 3
batch = 1
rdm = 82
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


人家训练是用预测胜率的模型来做的。。我对于预测胜率这种东西非常没有把握QAQ我打算预测棋路，希望人没事🙏

In [ ]:

class CNN(nn.Module):
  def __init__(self,ic,oc):
    super().__init__()
    self.conv=nn.Conv2d(
        ic,oc,3,stride=1,padding=1,dilation=1,
        groups=1,bias=False,padding_mode='zeros'
    )
    self.bn = nn.BatchNorm2d(oc)
    # 批量归一化
  def forward(self,x):
    y = self.conv(x)
    y = self.bn(y)
    y = torch.relu(y)
    # relu激活函数
    return y

class RSN(nn.Module):
  def __init__(self,ic,oc):
      super().__init__()
      self.conv_net = nn.Sequential(
          CNN(ic,oc),
          CNN(oc,ic)
      )
  def forward(self,x):
    x = self.conv_net(x) + x
    return x

class outputhdv1(nn.Module):
  def __init__(self,oc,hmc):
    super().__init__()
    self.cnn=CNN(oc,hmc)
    self.pL = nn.Conv2d(hmc,1,1)
  def forward(self,h):
    x = self.cnn(h)
    pl = self.pL(x)
    pl = pl.squeeze(1)
    return pl

class model(nn.Module):
  def __init__(self,b,f):
    super().__init__()
    self.name = "res"
    self.size = (b,f)
    self.iph = CNN(inputc,f)
    self.tru = nn.ModuleList()
    for i in range(b):
      self.tru.append(RSN(f,f))
    self.opt = outputhdv1(f,f)

  def forward(self,x):
    if(x.shape[1]==2):
      x = torch.cat((x,torch.zeros(batch,1,N,N,device=device)),dim=1)
    h = self.iph(x)
    for blk in self.tru :
      h = blk(h)
    h=self.opt(h)
    return h

def loadM(filep):
  if(filep=='stochastic'):
    return 'stochastic'
  model = torch.load(filep,map_location=torch.device(device))
  return model


这叫做棋盘+胜负判断一起搞定

In [ ]:
from ast import BoolOp
def getNeighbor(x):
  # print(x)
  neighbors = [ x-N, x+1-N, x+1, x+N, x-1+N, x-1]
  not_top = x >= N
  not_bottom = x < N*N-N
  not_left = x % N != 0
  not_right = x % N != N-1
  conditions = [ not_top, not_top and not_right, not_right, not_bottom, not_bottom and not_left, not_left ]
  i = 0
  q = []
  while i < 6:
    if(conditions[i]):
      q.append(neighbors[i])
    i+=1
  return q

class bd:
  def __init__(self):
    self.board = np.zeros(shape=(2,N*N))
    self.fa = np.zeros(shape=(2,N*N+2),dtype = int)
    self.stp = 0
    self.nc = 1

    for i in range(N*N + 2):
      self.fa[0][i]=self.fa[1][i]=i

    self.ru = N*N
    self.rd = N*N + 1
    for i in range(N):
      self.fa[0][i] = self.ru
      self.fa[0][N*N-1-i] = self.rd

    self.bu = N*N
    self.bd = N*N + 1
    for i in range(N):
      self.fa[1][i*N] = self.bu
      self.fa[1][i*N+N-1] = self.bd
  
  def getf(self,col,x):
    if self.fa[col][x]!=x:
      self.fa[col][x]=self.getf(col,self.fa[col][x])
    return self.fa[col][x]
  
  def merge(self,col,x,y):
    u = self.getf(col,x)
    v = self.getf(col,y)
    if u > v:
      u,v=v,u
    if u != v:
      self.fa[col][u]=v

  def win(self):
    if self.getf(0,self.ru) == self.getf(0,self.rd):
      return 0
    if self.getf(1,self.bu) == self.getf(1,self.bd):
      return 1
    
    return -1

  def at(self,x):
    if self.board[0][x] == 1:
      return 0
    if self.board[1][x] == 1:
      return 1
    return -1

  def play(self,x,col):
    self.board[col][x] = 1
    
    if(col == 0):
      self.nc = -1
    else :
      self.nc = 1

    self.stp += 1
    u = getNeighbor(x)
    for p in u :
      if self.at(p) == col:
        self.merge(col,p,x)
    
  
  def gen(self,t):
    nb = np.zeros(shape=(2,N,N))
    for i in range(2):
      for j in range(N*N):
        if(self.board[i][j]):
          if(not t):
            nb[i][j//N][j%N] = 1
          else :
            nb[1-i][j%N][j//N] = 1
    return nb

  def getcurrentplayer(self) -> int:
      return self.nc
    
  def getPossibleActions(self):
      acp = []
      for i in range(N*N):
        if self.at(i) == -1:
          acp.append((i,self.nc))
      # print(acp)
      # nn.exit()
      return acp

  def isTerminal(self)->bool:
      if self.win()==-1:
        return False
      return True

  def getReward(self)->float:
      return 1

  def takeAction(self, action: any):
      newstate = copy.deepcopy(self)
      newstate.play(action[0],0 if action[1] == 1 else 0)
      return newstate


In [ ]:
def show(board) -> None:
        for i in range(N):
            print(" " * i, end="")
            for j in range(N):
              if board[0][i][j] == 1:
                print("O",end = " ")
              else :
                if(board[1][i][j] == 1):
                  print("X",end= " ")
                else :
                  print("_",end= " ")
            print()
def show1(board) -> None:
        for i in range(N):
            print(" " * i, end="")
            for j in range(N):
              if board[0][i*N+j] == 1:
                print("O",end = " ")
              else :
                if(board[1][i*N+j] == 1):
                  print("X",end= " ")
                else :
                  print("_",end= " ")
            print()            
def show3(board:bd) -> None:
        for i in range(N):
            print(" " * i, end="")
            for j in range(N):
              if board.board[0][i*N+j] == 1:
                print("O",end = " ")
              else :
                if(board.board[1][i*N+j] == 1):
                  print("X",end= " ")
                else :
                  print("_",end= " ")
            print()            
def show2(board,p)->None:
          print(f"now is {p}")
          for i in range(N):
            print(" " * i, end="")
            for j in range(N):
                if(board.getf(p,i*N+j)!=i*N+j):
                  print("OX"[board.at(i*N+j)],end="")
                print(board.getf(p,i*N+j),end=" ")
            print(f"\n")

输入棋盘状态，当前操作人，找到移动点

nextmove2 先用按顺序下棋判断吧

In [ ]:
def tr(a):
  return a // N + (a % N) * N

def Nextmove1(board,model,flg)->int:
    
    
    if(flg):
      bd = np.array([np.split(board[1],N),np.split(board[0],N)])
      bd = bd.swapaxes(1,2)
    else:
      bd = np.array([np.split(board[0],N),np.split(board[1],N)])

    # print("what we see is : ")
    # show(bd)

    nnboard = torch.FloatTensor(bd)
    nnboard = nnboard.to(device)
    nnboard.unsqueeze_(0)
    p = model(nnboard)

    p = p.to('cpu')
    policy = p.detach().numpy().reshape((-1))
    
    for i in range(N*N):
      if (board[0][i] == 1) or (board[1][i] == 1): # 干掉非法棋步
        policy[i] = -1e4
    policy = policy - np.max(policy)

    for i in range(N*N):
      if policy[i]< -1:
        policy[i] = -1e4
    probs = np.exp(policy)
    probs = probs/probs.sum()
    # print(probs)
    action_space = int(np.random.choice([i for i in range(N*N)],p = probs))

    if(flg):
      action_space = tr(action_space) 
    return action_space

def Nextmove3(board,model,flg)->int:
  if flg == 1 :
    i = 5
    for j in range(N // 2 + N % 2):
      
        if(board[0][ i*N + 5 - j]!=1 and board[1][i*N + 5 - j]!=1):
          return i*N+ 5 - j
    for j in range(N // 2):
        if(board[0][i*N + j + 6]!=1 and board[1][i*N + j + 6]!=1):
          return i*N + j + 6

    for i in range(N):
      for j in range(N):
        if(board[0][i*N+j]!=1 and board[1][i*N+j]!=1):
          return i*N+j
  if(flg == 0):
    i = 5

    for j in range(N//2 + N % 2):
        if(board[0][(j+5)*N+i]!=1 and board[1][(j+5)*N+i]!=1):
          return (j+5)*N+i
    
    for j in range(N//2):
        if(board[0][j*N+i]!=1 and board[1][j*N+i]!=1):
          return j*N+i

    for i in range(N):
      for j in range(N):
        if(board[0][j*N+i]!=1 and board[1][j*N+i]!=1):
          return j*N+i

       
def Nextmove2(board,model,flg)->int:
    i = 0
    while i<100:
      r = random.randint(0,N*N-1)
      if board[0][r] != 1 and board[1][r] !=1:
        return r
      i +=1
    return Nextmove3(board,model,flg)


In [ ]:
def Nextmove4(board,model,flg)->int:
  show1(board)
  a = 1
  b = 2
  while board[0][a*N+b] == 1 or board[1][a*N+b] == 1:
    print("Input : ")
    a,b=input().split()
    a = int(a)
    b = int(b)
  print(a*N+b)
  return a*N+b

def Nextmove5(board:bd,mcts,nac):
  action = mcts.search(initialState = board)
  return action[0]

In [ ]:

class player:
  def __init__(self,model = 0,mcts = 0):
    self.flg = -1
    self.mcts = mcts
    if(model == 0):
      self.flg = 0
    if(model == 1):
      self.flg = 1 
    if(model == 2):
      self.flg = 2
    self.model = model
  def Nextmove(self,board,nac):
    if(self.flg == 1):
      return Nextmove4(board.board,0,nac)
    if(self.flg == 0):
      return Nextmove2(board.board,0,nac)
    if(self.flg == 2):
      return Nextmove5(board,self.mcts,nac)
    # if(random.randint(0,100)>rdm):
    #   # print("random!")
    #   return Nextmove2(board.board,0,nac)
    return Nextmove1(board.board,self.model,nac)
  # nac now active player


随机下完一整盘棋，然后如果有需要我们就随机下很多盘棋

pkonedata返回一个movelist，然后我们解析出这盘对局的信息。

In [ ]:
def pkonedata(p1,p2):
  tnb = bd()
  tnb.play(1*N+2,0)
  o = 1
  movelist = [1*N+2]
  while tnb.win() == -1:
      if(p2.flg==1):
        print("people")

      a = p2.Nextmove(tnb,(o&1))
      movelist.append(a)
      tnb.play(a,(o&1))
      o+=1

      if(p2.flg==1):
         show3(tnb)
      if tnb.win() != -1:
        break

      if(p2.flg==1):
        print("network")
      
      b = p1.Nextmove(tnb,(o&1))
      movelist.append(b)
      tnb.play(b,(o&1))
      o+=1

      if(p2.flg==1):
        show3(tnb)
      # show3(tnb)
  winner = tnb.win()
  # nn.exit()
  # print(movelist)
  # if(winner == 1):
  #   show2(tnb,0)
  #   show2(tnb,1)
  #   show3(tnb)
  #   nn.exit()
      # print(a,b)
  # print(movelist)
  return movelist,winner

def getpkdata(movelist,win,qs):
  mrc = []
  tbd = np.zeros(shape = (2,N,N),dtype = float)
  for id,nxt in enumerate(movelist):
    if (id & 1) == win:
        a = copy.deepcopy(tbd)
        a = a.astype(np.float32)
        tmp = np.zeros(shape=(N,N),dtype=float)
        tmp[nxt//N][nxt%N] = 1
        if (id & 1) == 1:
          b = np.asarray([a[1],a[0]])
          a = b.swapaxes(1,2)
          tmp = np.zeros(shape=(N,N),dtype=float)
          tmp[nxt%N][nxt//N] = 1
        mrc.append((a,tmp))     
        if qs == win : # 加权
          mrc.append((a,tmp))
    tbd[id&1][nxt//N][nxt%N] = 1
  return mrc

def gettotalpk(total,p1,p2,qs = -1):
    a = []
    for i in range(total):
      p = pkonedata(p1,p2)
      a += getpkdata(p[0],p[1],qs)
    return a

强化学习部分：

1. 实现一个评测胜率的东西qwq

2. 实现一个学习的过程，每次随机学。

In [ ]:

import torch.optim as optim
from torch.utils.data import DataLoader

def winningrating(n,p1,p2):
  rate = 0
  for i in range(n//2 + n%2):
    # print(i)
    rate += int(pkonedata(p1,p2)[1] == 0)
  for i in range(n//2):
    # print(i)
    rate += int(pkonedata(p2,p1)[1] == 1)
  rate /= n
  return rate

def fit(mymod:model,dataset,epoch):
  # nn.exit()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(mymod.parameters())
  
  L = len(dataset)
  print(f"train is {L}")

  for p in range(epoch):
    sum = 0
    for id,data in enumerate(dataset):
      input,label = data
      input = input.to(device)
      label = label.to(device)
      optimizer.zero_grad()
      output = mymod(input)
      loss = criterion(output,label)
      loss.backward()
      optimizer.step()
      sum += loss.item()

  print(f"now loss is {sum/L}")
  return mymod,sum/L

def check(data):
  print(f"qwq is {len(data)}")
  for id,data in enumerate(data):
    # print(i,j)
    i,j = data
    # print(j)
    # nn.exit(0)
    p = 0
    for k in range(N*N):
      if j[0][k] == 1.0:
        p = k
        break
    if i[0][0][p//N][p%N] == 1:
      print(id)
    else:
      print(-id)
  data = 0

def reinforcementlearning(n ,k , p1 , p2 , model1 ,model2 = 0,save=True,every = 3,numgames= 10):
  # model1.save("rsmodel.pth")
  winraterc = []
  for i in range(n//k):
    dataset1 = gettotalpk(k // 2 + k % 2, p1 ,p2)
    dataset2 = gettotalpk(k // 2 , p2 , p1)
    dataset = DataLoader(dataset1 + dataset2,batch_size = batch,shuffle = False)
    # check(dataset)
    # print(len(dataset))
    # nn.exit(0)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model1.parameters())
    L = len(dataset)
    print(f"train is {L}")
    sum = 0
    for id,data in enumerate(dataset):
      input,label = data
      input = input.to(device)
      label = label.to(device)
      print(input,label)
      optimizer.zero_grad()
      output = mymod(input)
      loss = criterion(output,label)
      loss.backward()
      optimizer.step()
      sum += loss.item()
    # if(model2 != 0):
    #   model2 = fit(model2,dataset)
    # if save:
    #   torch.save("rsmodel.pth")
    if i % every == 0:
      o = (winningrating(numgames,p1,p2),numgames)
      winraterc.append(o)
      print(f"after {i},winrate now is {o[0]}")
  return model1,model2,winraterc


加油吧。还差得多。先写个开始训练的模块

In [ ]:
# initialState = bd()

# print(initialState.isTerminal())



# print(action)

# nn.exit()

In [ ]:

mymod = loadM("/content/drive/MyDrive/Train/NR8/rsmodel_0.8.pth") # one 
youmod = loadM("/content/drive/MyDrive/Train/NR5/rsmodel_0.88.pth")
Qmod = loadM("/content/drive/MyDrive/Model/model5_32.0_0.06518.pth")

mymod.cuda()
youmod.cuda()
Qmod.cuda()


p1 = player(mymod)
p2 = player(youmod)
p3 = player()
p4 = player(2,mcts.mcts(timeLimit=1000))


loada = input()

peopletrain = 0
rdm = 88
ex = True
n = 160
k = 4
epoch = 4
every = 4
numgames = 25
mix = 1
mix2 = 0.6

# inita = winningrating(10,p4,p3)
# print(inita)
# nn.exit()

if peopletrain:
  p2 = player(1)
  epoch = 10
  k = 1
  n = 60

# p2 = p4

if not peopletrain:
  inita = winningrating(100,p1,p3)
  # initb = 0
  if p2.flg != -1:
    initb = winningrating(100,p2,p3)
  print(f"First round,winrate now is {inita},{initb}")

for i in range(n//k):

  dataset1 = gettotalpk(k // 2 + k % 2, p1 , p2)
  dataset2 = gettotalpk(k // 2 , p2 , p1)
  dataset3 = gettotalpk(1 , p1 , p4, 1)

  dataset = DataLoader(dataset1 + dataset2 + dataset3 + dataset3 ,batch_size = batch,shuffle = False)

  Qmod,sum = fit(Qmod,dataset,epoch)
  if (ex and random.randint(0,100)>78) or peopletrain:
    print("mymod")
    mymod,_ = fit(mymod,dataset,epoch-2)
    if p2.flg== -1:
      print("youmod")
      youmod,_ = fit(youmod,dataset,epoch-2)
    # escape too fast dropdown  

  if peopletrain:
      torch.save(mymod,f"/content/drive/MyDrive/Train/NR{loada}/rsmodel_{1}.pth")
      p1 = player(mymod)
      continue
  
  if(sum<mix or (i % every == 0 and i != 0)):
    o = (winningrating(numgames,player(Qmod),p3),numgames)
    print(f"In {sum:.5},winrate now is {o[0]}")
    if o[0] > mix2:
      print("saved")
      torch.save(Qmod,f"/content/drive/MyDrive/Train/NR{loada}/rsmodel_{o[0]}.pth")
    mix = sum
  
  if (i % (every+3) == 0 and i != 0):

    o = (winningrating(numgames,p1,p3),numgames)
    if p2.flg == -1:
      p = (winningrating(numgames,p2,p3),numgames)
    else:
      p = (0,0)
    print(f"after {i},p1 winrate now is {o[0]},p2 winrate now is {p[0]}")
    
    if o[0] > inita:
      p1 = player(mymod)
      inita = o[0]
      print("savedx")
      torch.save(mymod,f"/content/drive/MyDrive/Train/NR{loada}/rsmodel_{o[0]}.pth")
#/content/drive/MyDrive/Train/NR0
    if p[0] > initb and p2.flg == -1:
      p2 = player(youmod)
      initb = p[0]
      print("savedy")
      torch.save(youmod,f"/content/drive/MyDrive/Train/NR{loada}/rsmodel_{p[0]}.pth")

print("Finished Trainning")


8
First round,winrate now is 0.68,0.8
train is 297
now loss is 0.008534261334614546
mymod
train is 297
now loss is 0.03563510067773414
youmod
train is 297
now loss is 0.051347044011724643
In 0.0085343,winrate now is 0.44
train is 279
now loss is 0.006061870121388811
mymod
train is 279
now loss is 0.025865602797769137
youmod
train is 279
now loss is 0.02667669979075996
In 0.0060619,winrate now is 0.76
saved
train is 405
now loss is 0.0024123993981171126
mymod
train is 405
now loss is 0.05524845681781435
youmod
train is 405
now loss is 0.06254864056538748
In 0.0024124,winrate now is 0.76
saved
train is 284
now loss is 0.0077057143317152904
train is 343
now loss is 0.0017789041131471984
In 0.0017789,winrate now is 0.52
train is 294
now loss is 0.0016525569706237044
In 0.0016526,winrate now is 0.28
train is 422
now loss is 0.002167762732661866
mymod
train is 422
now loss is 0.043174514436242845
youmod
train is 422
now loss is 0.05007892518093833
train is 298
now loss is 0.00216965491774388

KeyboardInterrupt: ignored

**“人类的赞歌就是勇气的赞歌！”**